In [ ]:
#导入 selenium 第三方库
#仔细学这个项目
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
#专门用于解析 HTML/XML 文档的库 beautifulsoup4
import bs4 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver import ActionChains
import csv

In [ ]:
#指定驱动路径
service = Service(executable_path = r"./chromedriver.exe")        #Service 是一个类

In [ ]:
#创建浏览器对象
driver = webdriver.Chrome(service=service)

#打开指定网址
driver.get("https://www.bilibili.com/")
driver.maximize_window()
driver.add_cookie({             #cookie有四个关键属性必须要有, 就是下面这四个       找核心cookie就是看看哪个cookie的过期时间最长, 哪个安全级别最多
    'name': 'SESSDATA',
    'value': '8e65aa00%2C1782049040%2C18faa%2Ac2CjDuPrn9MJXNu_rlpME54SQoKJU2AwHp-VkJwDqH3-XW0BnAtll875CgQDJG9SKL4L8SVmRUZkxRd1FiSnBfVGpKMGdPMTBtTFdGTEk3ajd6b3owNlc4dFVocnV5RmJQYW82TDV6VG9FeWFkVzBTMUg5eDB4Q1Q0Y0NtcVk1eEVsOVRDSzlyamZnIIEC',
    'domain': '.bilibili.com',  # 核心：指定Cookie所属域名  必须是以.开头
    'path': '/',  # 核心：指定Cookie生效的路径（根路径）
    'httpOnly': True,  # 匹配B站SESSDATA的HttpOnly属性
    'secure': True,  # 匹配Secure属性（HTTPS下生效）
})
driver.refresh()  # 刷新页面，使Cookie生效

In [ ]:
# 设置要爬取的页数
total_page = 7
# 同样由于B站网页代码的更改，通过找到并点击下一页的方式个人暂不能实现  #妈的我也是
#（对，不会分析那个破网页！！！）
keyword = "部落冲突"

for i in range(0, total_page):

    url = (f"https://search.bilibili.com/video?keyword={keyword}"
               f"&page={i}")        #用&page= 来翻页!!!
 
    print(f"===========正在尝试获取第{i + 1}页网页内容===========")
    print(f"===========本次的url为：{url}===========")
    
    driver.get(url)
        # 这里请求访问网页的时间也比较久(可能因为我是macos)，所以是否需要等待因设备而异
        # 取消刷新并长时间休眠爬虫以避免爬取太快导致爬虫抓取到js动态加载源码
        # browser.refresh()
    print('正在等待页面加载：3')
    time.sleep(1)
    print('正在等待页面加载：2')
    time.sleep(1)
    print('正在等待页面加载：1')
    time.sleep(1)
    
# 直接分析网页
    html = driver.page_source
    # print("网页源码" + html) 用于判断是否获取成功
    soup = bs4.BeautifulSoup(html, 'lxml')      #lxml是最常用解析器的一种
    infos = soup.find_all(class_='bili-video-card')
    bv_id_list = []
    for info in infos:
            # 只定位视频链接
        href = info.find('a').get('href')
            # 拆分
        split_url_data = href.split('/')
            # 利用循环删除拆分出现的空白
        for element in split_url_data:
            if element == '':
                split_url_data.remove(element)
        # 打印检验内容
        # print(split_url_data)
        # 获取bvid
        bvid = split_url_data[2]        #b站视频链接格式为 bilili.com/video/BVxxxxxx
 
        # 利用if语句直接去重
        if bvid not in bv_id_list:
            bv_id_list.append(bvid)
            
        #打印出结果
    print(f"第{i + 1}页获取到的bvid列表为：{bv_id_list}")
    print(f"第{i + 1}页一共获取到{len(bv_id_list)}个视频bvid")
    
    
    #写文件很多都要用with
    with open(f"{keyword}.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["BVID"])  # 写入表头（可选，更清晰）
        for bvid in bv_id_list:
            writer.writerow([bvid])  # 逐行写入BVID